In [25]:
# Imports
import pandas as pd
import math
import time 

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split

In [26]:
# Constants
DATA_PATH = './data/'
DATASET_PATH = DATA_PATH + 'uci-data/'
MODELS_PATH = DATA_PATH + 'models/filtered-models/'
FEATURES = ['bodyAccX', 'gravityAccX', 'accY', 'gravityAccY', 'accZ', 'gravityAccZ', 'gyroX', 'gyroY', 'gyroZ', "subject", "activity"]

BATCH_SIZE = 32
SAMPLE_SIZE = BATCH_SIZE * 1500
EPOCHS = 5

In [27]:
complete_set = pd.read_csv(DATA_PATH + "self-calculated/complete-filtered.txt", sep='\s+', header=None)
complete_set.columns = FEATURES
train_set, test_set = train_test_split(complete_set, test_size=0.3)

<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
/var/folders/lt/w0169b7x5ml3psz3nly9vj3m0000gn/T/ipykernel_45211/4155984055.py:1: SyntaxWarning: invalid escape sequence '\s'
  complete_set = pd.read_csv(DATA_PATH + "self-calculated/complete-filtered.txt", sep='\s+', header=None)


In [28]:
# Creating model
device = None

if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

class RawDataModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.sequential_module = nn.Sequential(
            nn.Conv1d(in_channels=len(FEATURES)-1, out_channels=256, kernel_size=1),
            nn.ReLU(), 
            nn.MaxPool1d(1),
            nn.BatchNorm1d(256),

            nn.Conv1d(in_channels=256, out_channels=128, kernel_size=1),
            nn.ReLU(), 
            nn.MaxPool1d(1),
            nn.BatchNorm1d(128),

            nn.Conv1d(in_channels=128, out_channels=64, kernel_size=1),
            nn.ReLU(), 
            nn.MaxPool1d(1),
            nn.BatchNorm1d(64),
            
            nn.Flatten(),

            nn.Linear(64, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(),

            nn.Linear(512, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(),

            nn.Linear(128, 12),
            nn.Softmax()
        )

    def forward(self, x):
        return self.sequential_module(x)
    
model = RawDataModel().to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [29]:
# Training and testing function
def train(dataloader, model, loss_fn, optimizer):
    # Get batch num
    num_batches = len(dataloader.dataset) / BATCH_SIZE
    i = 0

    # Set the model to train mode
    model.train()

    for batch, (X, y) in enumerate(dataloader):
        X = X.unsqueeze(-1)
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        i += 1
        if batch % 50 == 0:
            print(f"loss: {loss.item()}, batch: {i} out of {math.ceil(num_batches)}")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X = X.unsqueeze(-1)
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    
    return(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [30]:
# Dataset preparation
class HAPTDataset(Dataset):
    def __init__(self, dataset, features, label):
        '''
        self.data = torch.tensor(dataset[features].values, dtype=torch.float32)[:SAMPLE_SIZE]
        self.labels = torch.tensor(dataset[label].values, dtype=torch.float32)[:SAMPLE_SIZE]
        '''
        max_size = len(dataset) - (len(dataset) % 32)
        self.data = torch.tensor(dataset[features].values, dtype=torch.float32)[:max_size]
        self.labels = torch.tensor(dataset[label].values, dtype=torch.float32)[:max_size]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]
    
train_dataset = HAPTDataset(train_set, train_set.columns[:-1], 'activity')
test_dataset = HAPTDataset(test_set, test_set.columns[:-1], 'activity')

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [31]:
EPOCHS = 5
perf_timer = time.perf_counter()
perf_acc = ""

for t in range(EPOCHS):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_function, optimizer)
    perf_acc = test(test_dataloader, model, loss_function)
    
print(perf_acc)
perf_timer = time.perf_counter() - perf_timer
print("Done!")

Epoch 1
-------------------------------
loss: 2.498300075531006, batch: 1 out of 17841


/opt/homebrew/lib/python3.12/site-packages/torch/nn/modules/module.py:1553: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


loss: 2.436281681060791, batch: 51 out of 17841
loss: 2.3011090755462646, batch: 101 out of 17841
loss: 2.2537169456481934, batch: 151 out of 17841
loss: 2.234772205352783, batch: 201 out of 17841
loss: 2.1094446182250977, batch: 251 out of 17841
loss: 2.035842180252075, batch: 301 out of 17841
loss: 2.1640610694885254, batch: 351 out of 17841
loss: 2.2120652198791504, batch: 401 out of 17841
loss: 1.959302544593811, batch: 451 out of 17841
loss: 2.167409658432007, batch: 501 out of 17841
loss: 2.1770434379577637, batch: 551 out of 17841
loss: 1.9949617385864258, batch: 601 out of 17841
loss: 2.059528350830078, batch: 651 out of 17841
loss: 1.9220120906829834, batch: 701 out of 17841
loss: 1.9277477264404297, batch: 751 out of 17841
loss: 2.0033063888549805, batch: 801 out of 17841
loss: 2.1151883602142334, batch: 851 out of 17841
loss: 2.0332794189453125, batch: 901 out of 17841
loss: 2.0315756797790527, batch: 951 out of 17841
loss: 1.946389079093933, batch: 1001 out of 17841
loss: 1

In [32]:
model_name = "cnn_4"
if True:
    torch.save(model.state_dict(), MODELS_PATH + model_name + ".pth")

    with open(MODELS_PATH + model_name + ".txt", "w") as f:
        f.write("Epochs: {}\n".format(EPOCHS))
        f.write("Feature Selection: {}\n".format("MRMR"))
        f.write("Feature Set: {}\n".format(FEATURES))
        f.write("Model: {}\n".format(str(model)))
        f.write("Loss Function: {}\n".format("Cross Entropy Loss"))
        f.write("Optimizer: {}\n\n\n\n".format(str(optimizer)))
        f.write("Results: {}\n".format(perf_acc))
        f.write("Timer: {}s\n".format(round(perf_timer, 2)))